In [18]:
import torch
torch.cuda.is_available()

True

In [2]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Reusing dataset imdb (C:\Users\ranax\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
imdb['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [14]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(3000))])
print(small_train_dataset)
# print(small_test_dataset[0])

Loading cached shuffled indices for dataset at C:\Users\ranax\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-8a9e43a6ac4acdff.arrow
Loading cached shuffled indices for dataset at C:\Users\ranax\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-2eff9f118d84c6fe.arrow


Dataset({
    features: ['text', 'label'],
    num_rows: 3000
})


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\ranax\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-7b8389fdf66d56b4.arrow
Loading cached processed dataset at C:\Users\ranax\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1\cache-4ad5845e9d9aba42.arrow


In [6]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [8]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
    load_accuracy = load_metric("accuracy")
    load_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "f1": f1}

In [15]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to C:\Users\ranax/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [16]:
# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "results"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch", 
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
c:\Study\Master Thesis\expaai_model\BertPatent\results is already a clone of https://huggingface.co/fassahat/results. Make sure you pull the latest changes with `repo.git_pull()`.


In [11]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 16


  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 0.6984978914260864,
 'eval_accuracy': 0.495,
 'eval_f1': 0.6622073578595318,
 'eval_runtime': 177.8841,
 'eval_samples_per_second': 16.865,
 'eval_steps_per_second': 1.057}

In [12]:
trainer.push_to_hub()

Saving model checkpoint to results
Configuration saved in results\config.json
Model weights saved in results\pytorch_model.bin
tokenizer config file saved in results\tokenizer_config.json
Special tokens file saved in results\special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.23k/3.23k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/fassahat/results
   ad602d4..f2ead03  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'dataset': {'name': 'imdb', 'type': 'imdb', 'config': 'plain_text', 'split': 'train', 'args': 'plain_text'}}
To https://huggingface.co/fassahat/results
   f2ead03..04971f1  main -> main



'https://huggingface.co/fassahat/results/commit/f2ead038e1be156332584d637febd5b717182e18'

In [13]:
from transformers import pipeline
 
sentiment_model = pipeline(model="fassahat/results")
sentiment_model(["I love this move", "This movie sucks!"])

loading configuration file https://huggingface.co/fassahat/results/resolve/main/config.json from cache at C:\Users\ranax/.cache\huggingface\transformers\f7dd29a7b1eeb7c41e11a8a77c73a4c762f0130330ee09f9fad54198005530a5.b7eece4ef1d7c7ff5ad9e7d8f0470b997ac530345183ea949655b950584452b5
Model config DistilBertConfig {
  "_name_or_path": "fassahat/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

loading configuration file https://huggingface.co/fassahat/results/resolve/main/config.json from cache at C:\Users\ranax/.cache\huggingface\transformers\f7dd29a7b1eeb7c41e11a8a77c73a4c762f0130330ee09f9fad54198005530a5.b7eece4ef1d7c7ff5ad9e7d8f0470b997ac530345183ea949655b950584452b5
Model config DistilBertConfig {
  "_name_or_path": "fassahat/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.21.1",
  "vocab_size": 30522
}

https://huggingface.co/fassahat/results/resolve/main/pytorch_model.bin not fou

storing https://huggingface.co/fassahat/results/resolve/main/pytorch_model.bin in cache at C:\Users\ranax/.cache\huggingface\transformers\cbd47f15e99366b67ee2f770e304cf3f48bcbd49d545349d9a584f93a2e2f378.f6515584913b5a094675c16e83714c1633666fab28aefe857e08b277f7d86148
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\cbd47f15e99366b67ee2f770e304cf3f48bcbd49d545349d9a584f93a2e2f378.f6515584913b5a094675c16e83714c1633666fab28aefe857e08b277f7d86148
loading weights file https://huggingface.co/fassahat/results/resolve/main/pytorch_model.bin from cache at C:\Users\ranax/.cache\huggingface\transformers\cbd47f15e99366b67ee2f770e304cf3f48bcbd49d545349d9a584f93a2e2f378.f6515584913b5a094675c16e83714c1633666fab28aefe857e08b277f7d86148
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at fassahat/results.
If your task is similar to the 

storing https://huggingface.co/fassahat/results/resolve/main/tokenizer.json in cache at C:\Users\ranax/.cache\huggingface\transformers\5a2f755e14b89c4d534ed8d915b350090b76aba251d3e61fa8bc2ade001ba5c1.f71e12dcf3314f964e59f54247509b88c99b9eac702db689a9c4bd9444c88904
creating metadata file for C:\Users\ranax/.cache\huggingface\transformers\5a2f755e14b89c4d534ed8d915b350090b76aba251d3e61fa8bc2ade001ba5c1.f71e12dcf3314f964e59f54247509b88c99b9eac702db689a9c4bd9444c88904
loading file https://huggingface.co/fassahat/results/resolve/main/vocab.txt from cache at C:\Users\ranax/.cache\huggingface\transformers\660d03915b1f97fe53cfa6c6bd1641a5e4a5fdc2ab89991649bde8a986f3113d.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/fassahat/results/resolve/main/tokenizer.json from cache at C:\Users\ranax/.cache\huggingface\transformers\5a2f755e14b89c4d534ed8d915b350090b76aba251d3e61fa8bc2ade001ba5c1.f71e12dcf3314f964e59f54247509b88c99b9eac702db689a9c4bd944

[{'label': 'LABEL_1', 'score': 0.5423027276992798},
 {'label': 'LABEL_1', 'score': 0.5513476729393005}]